Define the data structure using type annotations and offer avenues for data validation and conversion

In [1]:
!pip install pydantic[email]

   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   ------------------------- -------------- 194.6/307.7 kB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 307.7/307.7 kB 3.8 MB/s eta 0:00:00


In [15]:
from typing import List
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    code_postal: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr
    
class Owner(BaseModel):
    name: str
    email: EmailStr

class Entreprise(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    owner: Owner
    address: Address
    employees: conlist(Employee, min_length=2)
    website: HttpUrl

In [16]:
entreprise_instance = Entreprise(
    name="Vision business consulting",
    owner={
        "name": "Youssef",
        "email": "Youssef@vision-business-consulting.com"
    },
    address={
        "street": "105 Bd Abdelmoumen",
        "city": "Casablanca",
        "code_postal": "20000",
    },
    employees=[
        {
            "name": "BAILAL Ayoub",
            "position": "stagiaire",
            "email": "bailalayoub4905@gmail.com"
        },
        {
            "name": "Mohammed test",
            "position": "Manager",
            "email": "Mohamed@vision-business-consulting.com"
        }
    ],
    website="https://vision-business.consulting/fr/accueil/",
)

# Printing the instance
print(entreprise_instance)

name='Vision business consulting' owner=Owner(name='Youssef', email='Youssef@vision-business-consulting.com') address=Address(street='105 Bd Abdelmoumen', city='Casablanca', code_postal='20000') employees=[Employee(name='BAILAL Ayoub', position='stagiaire', email='bailalayoub4905@gmail.com'), Employee(name='Mohammed test', position='Manager', email='Mohamed@vision-business-consulting.com')] website=Url('https://vision-business.consulting/fr/accueil/')


In [3]:
print(entreprise_instance.model_fields_set)

{'name', 'address', 'website', 'employees', 'owner'}


In [4]:
print(entreprise_instance.model_dump())
print(entreprise_instance.model_dump_json())
print(entreprise_instance.model_json_schema())

{'name': 'Vision business consulting', 'owner': {'name': 'Youssef', 'email': 'Youssef@vision-business-consulting.com'}, 'address': {'street': '105 Bd Abdelmoumen', 'city': 'Casablanca', 'code_postal': 20000}, 'employees': [{'name': 'BAILAL Ayoub', 'position': 'stagiaire', 'email': 'bailalayoub4905@gmail.com'}, {'name': 'Mohammed test', 'position': 'Manager', 'email': 'Mohamed@vision-business-consulting.com'}], 'website': Url('https://vision-business.consulting/fr/accueil/')}
{"name":"Vision business consulting","owner":{"name":"Youssef","email":"Youssef@vision-business-consulting.com"},"address":{"street":"105 Bd Abdelmoumen","city":"Casablanca","code_postal":20000},"employees":[{"name":"BAILAL Ayoub","position":"stagiaire","email":"bailalayoub4905@gmail.com"},{"name":"Mohammed test","position":"Manager","email":"Mohamed@vision-business-consulting.com"}],"website":"https://vision-business.consulting/fr/accueil/"}
{'$defs': {'Address': {'properties': {'street': {'title': 'Street', 'type

Field validator

In [5]:
from pydantic import field_validator

class Stagiaire(BaseModel):
    name: str
    email: EmailStr
    
    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('Owner name must contain a space')
        return v.title()

try:
    stagiaire_instance = Stagiaire(name="bailal ayoub", email="bailalayoub4905@gmail.com")
except ValueError as e:
    print(e)

print(stagiaire_instance)

name='Bailal Ayoub' email='bailalayoub4905@gmail.com'


Model validation

In [6]:
from typing import Any
from pydantic import BaseModel, EmailStr, ValidationError, model_validator

class Stagiaire(BaseModel):
    name: str
    email: EmailStr
    
    @model_validator(mode='before')
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError('password should not be included')
            if 'card_number' in data:
                raise ValueError('card_number should not be included')
        return data
    
    @model_validator(mode='after')
    def check_name_contains_space(self) -> 'Stagiaire':
        if ' ' not in self.name:
            raise ValueError('Stagiaire name must contain a space')
        return self
 
try:
    stagiaire_instance = Stagiaire(name="bailalayoub", email="bailalayoub4905@gmail.com", password="password123")
except ValidationError as e:
    print(e) 


1 validation error for Stagiaire
  Value error, password should not be included [type=value_error, input_value={'name': 'bailalayoub', '...assword': 'password123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error


Field - Used to customize and add metadata to fields of models

In [7]:
from typing import List
from pydantic import BaseModel, Field, EmailStr
from decimal import Decimal

class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=10)
    email: EmailStr = Field(...)
    age: int = Field(..., gt=0, le=120)
    height: float = Field(..., gt=0.0)
    is_active: bool = Field(True)
    favorite_numbers: List[int] = Field(..., min_items=1)

In [8]:
user_instance = User(
    username="bailal",
    age=23,
    height=6.1,
    weight=98.5,
    email="bailalayoub4905@gmail.com",
    password="securepassword",
    favorite_numbers=[5, 9]
)

print(user_instance)

username='bailal' email='bailalayoub4905@gmail.com' age=23 height=6.1 is_active=True favorite_numbers=[5, 9]


Computed field - strict mode

In [34]:
from pydantic import BaseModel, ValidationError, field_validator, computed_field
from datetime import datetime

class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field
    @property
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year

    @field_validator('birth_year')
    @classmethod
    def validate_age(cls, v: int) -> int:
        current_year = datetime.now().year
        if current_year - v < 18:
            raise ValueError('Person must be 18 years or older')
        return v

try:
    print(Person.model_validate({'name' : 'Ayoub', 'birth_year' : '2000'}, strict=True))
except ValidationError as e:
    print(e)

1 validation error for Person
birth_year
  Input should be a valid integer [type=int_type, input_value='2000', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/int_type


## Pydantic Settings

In [10]:
!pip install pydantic-settings